In [42]:
import re, string
import random
import json as json
from nltk import word_tokenize
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier

stop_words = stopwords.words('english')

with open('master.json') as f:
    palantir = json.load(f)
    
def tweets_to_string(data):    
    list = []   
    for key in data:
        list += [data[key]['text']]
    return list
        
palantir_tweets = tweets_to_string(palantir)

positive_tweets = twitter_samples.strings('positive_tweets.json')

negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
palantir_tweet_tokens = [word_tokenize(i) for i in palantir_tweets]

    

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

def remove_noise(tweet_tokens, stop_words = ()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []
palantir_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
    
for tokens in palantir_tweet_tokens:
    palantir_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
    
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)
all_neg_words = get_all_words(negative_cleaned_tokens_list)
all_palantir_words = get_all_words(palantir_cleaned_tokens_list)

#freq_dist_pos = FreqDist(all_pos_words)
#print(freq_dist_pos.most_common(10))

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)
palantir_tokens_for_model = get_tweets_for_model(palantir_cleaned_tokens_list)

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

palantir_positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in palantir_tokens_for_model]

palantir_negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in palantir_tokens_for_model]

dataset = positive_dataset + negative_dataset
palantir_dataset = palantir_positive_dataset + palantir_negative_dataset

random.shuffle(dataset)
random.shuffle(palantir_dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, palantir_dataset))

print(classifier.show_most_informative_features(100))

Accuracy is: 0.655
Most Informative Features
                      :) = True           Positi : Negati =    990.5 : 1.0
                follower = True           Positi : Negati =     35.7 : 1.0
                  arrive = True           Positi : Negati =     30.4 : 1.0
                     sad = True           Negati : Positi =     22.9 : 1.0
                     bam = True           Positi : Negati =     19.0 : 1.0
                     x15 = True           Negati : Positi =     16.3 : 1.0
                      aw = True           Negati : Positi =     16.3 : 1.0
                     bro = True           Positi : Negati =     14.4 : 1.0
               community = True           Positi : Negati =     14.4 : 1.0
                  friday = True           Positi : Negati =     13.0 : 1.0
                    blog = True           Positi : Negati =     12.4 : 1.0
                 welcome = True           Positi : Negati =     11.2 : 1.0
                     via = True           Positi : Nega

In [93]:
import re, string
import random
import json as json
from nltk import word_tokenize
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from textblob import TextBlob

#get the tweets from the json
with open('master.json') as f:
    palantir = json.load(f)
    
def tweets_to_string(data):    
    list = []   
    for key in data:
        list += [data[key]['text']]
    return list
def retweet_count(data):
    list = []   
    for key in data:
        list += [data[key]['retweet_count']]
    return list
def favorite_count(data):
    list = []   
    for key in data:
        list += [data[key]['favorite_count']]
    return list

palantir_tweets = tweets_to_string(palantir) #array of related tweets
retweets = retweet_count(palantir)
favorites = favorite_count(palantir)

#print(palantir_tweets)

# calculate the sentiment for each tweet 
scores = [] # array that stores the sentiment value (neg, neu, pos, compound) 
            # where compound is the aggregate sentiment
for tweet in palantir_tweets:
    score = analyser.polarity_scores(tweet)
    scores.append(score)
    
# print the tweet and the associated score
# i = 0
# for i in range(len(palantir_tweets)):
    # print(str(i+1) + ".json: "+ palantir_tweets[i] + ", score = " + json.dumps(scores[i]))

txtblob = []
for tweet in palantir_tweets:
    #print(TextBlob(tweet).sentiment.polarity)
    txtblob.append(TextBlob(tweet).sentiment.polarity)
    
def sentiment_analyzer_scores(text):
    score = analyser.polarity_scores(text)
    lb = score['compound']
    if lb >= 0.05:
        return 1
    elif (lb > -0.05) and (lb < 0.05):
        return 0
    else:
        return -1
    
vader = []
for tweet in palantir_tweets:
    tmp = sentiment_analyzer_scores(tweet) 
    vader.append(tmp)
    
#Converting List of Dictionaries into Dataframe
dataFrame = pd.DataFrame(scores)
dataFrame['vader'] = vader
dataFrame['txtblob'] = txtblob
dataFrame['retweets'] = retweets
dataFrame['favorites'] = favorites
print(dataFrame)
#dataFrame.mean()



      neg    neu    pos  compound  vader   txtblob  retweets  favorites
0    0.00  0.759  0.241    0.5859      1  0.600000         0          0
1    0.00  0.794  0.206    0.6369      1  0.200000         1          6
2    0.00  0.854  0.146    0.3400      1  0.000000         0          0
3    0.00  0.806  0.194    0.4310      1  0.333333         0          0
4    0.00  1.000  0.000    0.0000      0  0.000000         0          1
..    ...    ...    ...       ...    ...       ...       ...        ...
195  0.00  1.000  0.000    0.0000      0  0.100000         0          0
196  0.00  1.000  0.000    0.0000      0  0.500000         0          0
197  0.11  0.890  0.000   -0.3818     -1 -0.500000         6         31
198  0.00  0.905  0.095    0.2960      1  0.000000         0          0
199  0.00  0.586  0.414    0.5473      1  0.800000         0          0

[200 rows x 8 columns]
